<a href="https://colab.research.google.com/github/vladimir-shvets/C-projects/blob/master/Copy_of_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My\ Drive/CV2/Exercises/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/CV2/Exercises


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jun 28 14:17:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import all necessary libraries

In [3]:
from __future__ import print_function
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset
import os
import imageio
import cv2
import numpy as np

## 1. Encoder-decoder network for saliency prediction

In [4]:
vgg16_conv = torch.load('vgg16-conv.pth')
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
torch.equal(vgg16_conv['conv1_1_weight'], next(iter(vgg16.parameters())))

True

### Encoder


In [6]:
vgg16 = torchvision.models.vgg16(pretrained=True)

for param in vgg16.parameters(): 
  param.requires_grad = False

encoder = vgg16.features[:-1]

### Decoder

In [7]:
decoder = nn.Sequential(
          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),

          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),

          nn.Conv2d(512, 256, 3, padding=1), nn.ReLU(),
          nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
          nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),

          nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
          nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),

          nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
          nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
          nn.Conv2d(64, 1, 1, padding=0)
        )

### Encoder-Decoder

In [9]:
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x 

## Loss function

In [10]:
class BCELossWithDownsampling():
  def __init__(self):
    self.downsample = nn.AvgPool2d(4, stride=4, count_include_pad=False)
    self.loss_fn = nn.BCEWithLogitsLoss()

  def __call__(self, pred, y):
    return self.loss_fn(self.downsample(pred), self.downsample(y))

## 3. Training the encoder-decoder network

### Data pre-processing

In [12]:
def read_file(filename):
	lines = []
	with open(filename, 'r') as file:
	    for line in file: 
	        line = line.strip() #or some other preprocessing
	        lines.append(line)
	return lines

class FixationDataset(Dataset):
	def __init__(self, root_dir, image_file, fixation_file, transform=None):
		self.root_dir = root_dir
		self.image_files = read_file(image_file)
		self.fixation_files = read_file(fixation_file)
		self.transform = transform
		assert(len(self.image_files) == len(self.fixation_files))

	def __len__(self):
		return len(self.image_files)

	def __getitem__(self, idx):
		if torch.is_tensor(idx):
			idx = idx.tolist()
	 
		img_name = os.path.join(self.root_dir, self.image_files[idx])
		image = imageio.imread(img_name)

		fix_name = os.path.join(self.root_dir, self.fixation_files[idx])
		fix = imageio.imread(fix_name)

		sample = {'image': image, 'fixation': fix}

		if self.transform:
			sample = self.transform(sample)

		return sample

class Rescale():
    '''Recaling data'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image, fixation = sample['image'], sample['fixation']
        image = image.astype(np.float32) / 255.0
        fixation = fixation.astype(np.float32) / 255.0
        return {'image': image, 'fixation': fixation}

class ToTensor():
    '''Convert data to a tensor'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image, fixation = sample['image'], sample['fixation']
        image = torch.from_numpy(np.transpose(image))
        fixation = torch.from_numpy(np.expand_dims(fixation, axis=0))
        return {'image': image, 'fixation': fixation}

class Normalize():
    '''Normalize tensor'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image, fixation = sample['image'], sample['fixation']
        normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return {'image': normalize(image), 'fixation': fixation}

### Training step

In [13]:
def get_dl(bs=20):
    transform = torchvision.transforms.Compose([Rescale(), ToTensor(), Normalize()])
    train_ds = FixationDataset('cv2_training_data', 'cv2_training_data/train_images.txt', 'cv2_training_data/train_fixations.txt', transform)
    valid_ds = FixationDataset('cv2_validation_data', 'cv2_validation_data/val_images.txt', 'cv2_validation_data/val_fixations.txt', transform)

    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
    return train_dl, valid_dl

In [14]:
def train(train_dl, valid_dl, model, loss_fn, opt, num_epoch):
  for epoch in range(num_epoch):
    print("training")
    model.train()
    train_loss = 0;
    for batch in train_dl:
      
      pred = model(batch['image'].to(device))
      loss = loss_fn(pred, batch['fixation'].to(device))
      train_loss += loss

      loss.backward()
      opt.step()
      opt.zero_grad()
    train_loss /= len(train_dl)

    model.eval()
    with torch.no_grad():
      valid_loss = sum(loss_fn(model(batch['image'].to(device)), batch['fixation'].to(device)) for batch in valid_dl)
    valid_loss /= len(valid_dl)
    print(f'Epoch {epoch: 2d}, training loss = {train_loss: .4f}, validation loss = {valid_loss: .4f}') 
    
    if epoch == num_epoch-1:
      PATH = f'autoencoder_epoch_Exercise5_{epoch}_Adam_1e-6.ckpt'
      torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'train_loss': train_loss,
                'valid_loss': valid_loss
                  }, PATH)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Generator().to(device)
loss_fn = BCELossWithDownsampling()
opt = torch.optim.Adam(model.parameters(), lr=1e-6,weight_decay=1e-4)
# opt = torch.optim.SGD(model.parameters(), lr=1e-2) 
train_dl, valid_dl = get_dl()

In [ ]:
# train
train(train_dl, valid_dl, model, loss_fn, opt, 15)

training


RuntimeError: ignored

# Load Model (if necessary)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Generator().to(device)
opt = torch.optim.SGD(model.parameters(), lr=1e-4,weight_decay=1e-4) 
checkpoint = torch.load('autoencoder_epoch_Exercise5_14_Adam_1e-4.ckpt')


model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])
curr_epoch_backup = checkpoint['epoch']
train_loss_backup = checkpoint['train_loss']
valid_loss_backup = checkpoint['valid_loss']

# Test dataset

In [16]:
def get_test_dl(bs=20):
    transform = torchvision.transforms.Compose([TestDatasetRescale(), TestToTensor(), TestNormalize()])
    test_ds = TestDataset('cv2_testing_data','cv2_testing_data/test_images.txt', transform)

    test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True,num_workers=6)
    return test_dl

In [17]:
class TestDataset(Dataset):
	def __init__(self, root_dir, image_file, transform=None):
		self.root_dir = root_dir
		self.image_files = read_file(image_file)
		self.transform = transform

	def __len__(self):
		return len(self.image_files)

	def __getitem__(self, idx):
		if torch.is_tensor(idx):
			idx = idx.tolist()
	 
		img_name = os.path.join(self.root_dir, self.image_files[idx])
		image = imageio.imread(img_name)

		sample = {'image': image}

		if self.transform:
			sample = self.transform(sample)

		return sample

class TestDatasetRescale():
    '''Recaling data'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image = sample['image']
        image = image.astype(np.float32) / 255.0
        return {'image': image}

class TestToTensor():
    '''Convert data to a tensor'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image = sample['image']
        image = torch.from_numpy(np.transpose(image))
        return {'image': image}

class TestNormalize():
    '''Normalize tensor'''
    def __init__(self):
        pass
    def __call__(self, sample):
        image = sample['image']
        normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return {'image': normalize(image)}



## Pre-processing data
Reference:
https://github.com/immortal3/MLNet-Pytorch

In [19]:
import os 
from sklearn.utils import shuffle

imgs_train_path = 'cv2_training_data/images/train'
maps_train_path = 'cv2_training_data/fixations/train'

imgs_val_path = 'cv2_validation_data/images/validation/'
maps_val_path = 'cv2_validation_data/fixations/validation/'



def generator(b_s, phase_gen='train'):
    if phase_gen == 'train':
        images = [imgs_train_path + f for f in os.listdir(imgs_train_path) if f.endswith('.png')]
        maps = [maps_train_path + f for f in os.listdir(maps_train_path) if f.endswith('.png')]
    elif phase_gen == 'val':
        images = [imgs_val_path + f for f in os.listdir(imgs_val_path) if f.endswith('.png')]
        maps = [maps_val_path + f for f in os.listdir(maps_val_path) if f.endswith('.png')]
    else:
        raise NotImplementedError

    images.sort()
    maps.sort()
    
    images , maps = shuffle(images,maps)
    
    counter = 0
    
    while True:
        yield preprocess_images(images[counter:counter + b_s], shape_r, shape_c), preprocess_maps(maps[counter:counter + b_s], shape_r_gt, shape_c_gt)
        if counter + b_s >= len(images):
          break
        counter = counter + b_s

def preprocess_images(paths, shape_r, shape_c):
    ims = np.zeros((len(paths), shape_r, shape_c, 3))

    for i, path in enumerate(paths):
        original_image = cv2.imread(path)
        padded_image = padding(original_image, shape_r, shape_c, 3)
        ims[i] = padded_image.astype('float')
    
#     cv2 : BGR
#     PIL : RGB
    ims = ims[...,::-1]
    ims /= 255.0
    ims = np.rollaxis(ims, 3, 1)  
    return ims


def preprocess_maps(paths, shape_r, shape_c):
    ims = np.zeros((len(paths), 1, shape_r, shape_c))

    for i, path in enumerate(paths):
        original_map = cv2.imread(path, 0)
        padded_map = padding(original_map, shape_r, shape_c, 1)
        ims[i, 0] = padded_map.astype(np.float32)
        ims[i, 0] /= 255.0
        
    return ims

def padding(img, shape_r=224, shape_c=224, channels=3):
    img_padded = np.zeros((shape_r, shape_c, channels) , dtype=np.uint8)
    if channels == 1:
        img_padded = np.zeros((shape_r, shape_c), dtype=np.uint8)

    original_shape = img.shape
    rows_rate = original_shape[0]/shape_r
    cols_rate = original_shape[1]/shape_c

    if rows_rate > cols_rate:
        new_cols = (original_shape[1] * shape_r) // original_shape[0]
        img = cv2.resize(img, (new_cols, shape_r))
        if new_cols > shape_c:
            new_cols = shape_c
        img_padded[:, ((img_padded.shape[1] - new_cols) // 2):((img_padded.shape[1] - new_cols) // 2 + new_cols)] = img
    else:
        new_rows = (original_shape[0] * shape_c) // original_shape[1]
        img = cv2.resize(img, (shape_c, new_rows))
        if new_rows > shape_r:
            new_rows = shape_r
        img_padded[((img_padded.shape[0] - new_rows) // 2):((img_padded.shape[0] - new_rows) // 2 + new_rows), :] = img

    return img_padded

# test_maps directory




## Removing

In [ ]:
# removing
! rm -rf test_maps


### Creating

In [ ]:
# creating
! mkdir test_maps

## Generate test_maps
Reference: https://github.com/immortal3/MLNet-Pytorch

In [20]:
shape_r = 224
shape_c = 224
test_dl = get_test_dl()

import time
import torchvision.transforms as transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

test_file = [ f for f in os.listdir('cv2_testing_data/images/test/') if f.endswith('.png') ]

for i in test_file:
    img = preprocess_images(['cv2_testing_data/images/test/'+i],shape_r,shape_c)
    img = torch.tensor(img.copy(),dtype=torch.float)
    for idx,x in enumerate(img):
        img[idx] = normalize(x)
    img = img.cuda()
    
    a = torch.sigmoid(model(img))
    pred = a*255.0
    # print(pred[0])
    # print("$$$$$$")
    cv2.imwrite('test_maps/' + 'prediction-' + i[6:-4] + '.png',pred[0].squeeze(0).data.cpu().numpy())
